## Importing all required packages.

In [1]:
# Importing required packages.
import pandas as pd
import numpy as np
#import pgeocode
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests # library to handle requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Making the DataFrame.


In [2]:
# Using the website link given and making a DataFrame our of it.
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
Neighbourhood = pd.DataFrame(dfs[0])

#Displaying DataFrame
Neighbourhood.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Let's change the string 'Not assigned' to __NaN__ so it can be removed from DataFrame.

In [3]:
#Chaning 'Not assigned' to NaN.
Neighbourhood.replace('Not assigned', np.NaN, inplace=True)

#Dropping NaN values.
Neighbourhood.dropna(inplace=True)

#Displaying final result.
Neighbourhood.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Let's see how many rows and columns we have in the end.

In [4]:
Neighbourhood.shape

(103, 3)

Let's add Latitude and Longitude of neighbourhoods to our DataFrame.

In [5]:
Neighbourhood_coordinates = pd.read_csv("http://cocl.us/Geospatial_data")

#Merging two DataFrames.
merged_neighbour = merged_car = pd.merge(Neighbourhood_coordinates, Neighbourhood, on='Postal Code')
merged_neighbour = merged_neighbour[['Postal Code','Borough', 'Neighborhood', 'Latitude','Longitude']]

#Displaying final result.
merged_neighbour.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Creating a map of Toronto

Get Toronto coordinates

In [6]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [7]:
# create map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_neighbour['Latitude'], merged_neighbour['Longitude'], merged_neighbour['Borough']\
                                           , merged_neighbour['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Downtown Toronto - preprocessing data

I decided to explore Downtown Toronto. Firstly, I create the another dataframe for it.

In [8]:
downtown_df = merged_neighbour[merged_neighbour['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Get Downtown coordinates

In [9]:
address = 'Toronto, Downtown Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


Creating the map of Toronto Downtown

In [10]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_df['Latitude'], downtown_df['Longitude'],downtown_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

## Explore Neighborhoods in Toronto Downtown

I use the Foursquare API to explore the neighborhoods and segment them.

In [11]:
CLIENT_ID = 'EHJAOYBBN3OFIBFP2D3MSC3W0XEP0PQ410OIFUISK0P42JZY' 
CLIENT_SECRET = 'JSQOOEGOFCG435OITCYKUM5ICMQVZA3PPOKRZAUUCSYA4XXU' 
VERSION = '20180605' 

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Function for getting venues

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create the dataframe for Toronto Downtown using my function

In [13]:
downtown_venues = getNearbyVenues(names=downtown_df['Neighborhood'],
                                  latitudes=downtown_df['Latitude'],
                                  longitudes=downtown_df['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [14]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


Find out how many unique categories can be curated from all the returned venues

In [15]:
len(downtown_venues['Venue Category'].unique())

214

## Analyze Each Neighborhood

In [16]:
# one hot encoding
downtn_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtn_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

downtn_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Examine the dataframe size.

In [17]:
downtn_onehot.shape

(1222, 214)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
downtn_grouped = downtn_onehot.groupby('Neighborhood').mean().reset_index()
downtn_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0000,0.0,...,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0000,0.0000,0.0,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0000,0.0,0.000000
2,Central Bay Street,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0000,0.0,...,0.0,0.0,0.015625,0.0,0.0,0.015625,0.0000,0.0000,0.0,0.015625
3,Christie,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0000,0.0,0.000000
4,Church and Wellesley,0.0125,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0125,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0125,0.0125,0.0,0.025000


Print each neighborhood along with the top 5 most common venues

In [19]:
num_top_venues = 5

for hood in downtn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtn_grouped[downtn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1                Café  0.03
2  Seafood Restaurant  0.03
3              Bakery  0.03
4          Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2     Airport Terminal  0.12
3  Rental Car Location  0.06
4   Airport Food Court  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.06
2      Sandwich Place  0.06
3                Café  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3    Candy Store  0.06
4      Nightclub  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.05
3           Restaura

A function to sort the venues in descending order.

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtn_grouped['Neighborhood']

for ind in np.arange(downtn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Cheese Shop,Restaurant,Bakery,Café,Eastern European Restaurant,Nightclub
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Rental Car Location,Bar,Plane,Sculpture Garden
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Japanese Restaurant,Thai Restaurant,Department Store,Burger Joint,Bubble Tea Shop
3,Christie,Grocery Store,Café,Park,Diner,Baby Store,Restaurant,Candy Store,Italian Restaurant,Athletics & Sports,Nightclub
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Pub,Hotel,Café,Yoga Studio


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 3 clusters.

In [22]:
# set number of clusters
kclusters = 3

downtn_grouped_clustering = downtn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [23]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtn_merged = downtown_df
downtn_merged = downtn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtn_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Pizza Place,Restaurant,Pub,Italian Restaurant,Café,Bakery,Beer Store,Park,Deli / Bodega
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Pub,Hotel,Café,Yoga Studio
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Café,Theater,Spa,Ice Cream Shop,Farmers Market
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Tea Room,Bookstore


Finally, let's visualize the resulting clusters

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtn_merged['Latitude'], downtn_merged['Longitude'], downtn_merged['Neighborhood'], downtn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

__Cluster 1 - residential areas__

In [25]:
downtn_merged.loc[downtn_merged['Cluster Labels'] == 0, downtn_merged.columns[[1] + list(range(3, downtn_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,43.679563,-79.377529,0,Park,Playground,Trail,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


__Cluster 2 - harbor__

In [26]:
downtn_merged.loc[downtn_merged['Cluster Labels'] == 1, downtn_merged.columns[[1] + list(range(3, downtn_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,43.667967,-79.367675,1,Coffee Shop,Pizza Place,Restaurant,Pub,Italian Restaurant,Café,Bakery,Beer Store,Park,Deli / Bodega
2,Downtown Toronto,43.665860,-79.383160,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Pub,Hotel,Café,Yoga Studio
3,Downtown Toronto,43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Breakfast Spot,Café,Theater,Spa,Ice Cream Shop,Farmers Market
4,Downtown Toronto,43.657162,-79.378937,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Tea Room,Bookstore
5,Downtown Toronto,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Gym,Hotel,Creperie,Lingerie Store,Moroccan Restaurant
6,Downtown Toronto,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Cheese Shop,Restaurant,Bakery,Café,Eastern European Restaurant,Nightclub
7,Downtown Toronto,43.657952,-79.387383,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Japanese Restaurant,Thai Restaurant,Department Store,Burger Joint,Bubble Tea Shop
8,Downtown Toronto,43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Hotel,Thai Restaurant,Clothing Store,Steakhouse,American Restaurant
9,Downtown Toronto,43.640816,-79.381752,1,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Scenic Lookout,Brewery,Sporting Goods Shop,Fried Chicken Joint,Baseball Stadium
10,Downtown Toronto,43.647177,-79.381576,1,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Seafood Restaurant,Japanese Restaurant,American Restaurant,Italian Restaurant,Beer Bar


__Cluster 3 - business center__

In [27]:
downtn_merged.loc[downtn_merged['Cluster Labels'] == 2, downtn_merged.columns[[1] + list(range(3, downtn_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,43.628947,-79.39442,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Rental Car Location,Bar,Plane,Sculpture Garden
